# Arash SC create a project for head position



This notebook demonstrates the necessary steps to use python and DeepLabCut for creating and analysins videos from orientation 
experiment 

This notebook illustrates how to:
 
- make a movie from images 
- create a project
- extract training frames
- label the frames
- plot the labeled images (optional)
- create a training set
- train a network
- evaluate a network
- analyze a novel video
- create an automatically labeled video (optional)
- Go to Matlab 



## Create a new project

It is always good idea to keep the projects seperate. This function creates a new project with subdirectories and a basic configuration file in the user defined directory otherwise the project is created in the current working directory.

You can always add new videos to the project at any stage of the project. 

In [1]:
import deeplabcut
## https://github.com/AlexEMG/DeepLabCut

In [ ]:
task='ar21_07_06_20LR_R' # Enter the name of your experiment Task
task='ar32_10_09_20Face3' 

experimenter='Arash' # Enter the name of the experimenter
Mainfolder =  'D:\\videos\\AR19distance\\2020_09_14_ 215321'; # Enter the name of the folder with images folders inside 
Mainfolder = 'D:\\videos\\ar32\\2020_10_09\\';

filenames = ['10.avi','18.avi','19.avi','22.avi']; # Enter the name of the files you want to train
video = [str(Mainfolder)+str(f) for f in filenames]; # add path to each file
print(video)
#deeplabcut.create_new_project(task,experimenter,video, working_directory=Mainfolder,copy_videos=False) #change the working directory to where you want the folders created.
deeplabcut.create_new_project(task,experimenter, video,copy_videos=True, multianimal=False)
#deeplabcut.create_new_project(task,experimenter, video,copy_videos=True)

## Extract frames from videos 
A key point for a successful feature detector is to select diverse frames, which are typical for the behavior you study that should be labeled.

This function selects N frames either uniformly sampled from a particular video (or folder) (algo=='uniform'). Note: this might not yield diverse frames, if the behavior is sparsely distributed (consider using kmeans), and/or select frames manually etc.

Also make sure to get select data from different (behavioral) sessions and different animals if those vary substantially (to train an invariant feature detector).

Individual images should not be too big (i.e. < 850 x 850 pixel). Although this can be taken care of later as well, it is advisable to crop the frames, to remove unnecessary parts of the frame as much as possible.

Always check the output of cropping. If you are happy with the results proceed to labeling.

In [ ]:
%matplotlib inline
path_config_file = 'D:\\Dropbox\\Notebook\\ar19_09_15_20Face2-Arash-2020-09-15\\config.yaml';
path_config_file = 'D:\\Dropbox\\Notebook\\ar32_10_09_20Face3-Arash-2020-10-10\\config.yaml';

#Enter the path of the config file that was just created from the above step (check the folder)
deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=False) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#deeplabcut.extract_frames(path_config_file,'automatic','uniform',crop=True, checkcropping=True) #there are other ways to grab frames, such as by clustering 'kmeans'; please see the paper. 
#You can change the cropping to false, then delete the checkcropping part!

## define the config path and load deeplabcut


In [2]:
# write the Path config file path
path_config_file = 'E:\\movies\\ar2breathing\\10_08_19\\ar2breathing_10_08_19LR_R-Arash-2020-04-30\\config.yaml';
path_config_file = 'D:\\videos\\ar21\\2020-07-06\\ar21_07_06_20Face2-Arash-2020-07-07\\config.yaml';
path_config_file = 'D:\\Dropbox\\Notebook\\ar19_09_15_20Face3-Arash-2020-09-16\\config.yaml';
path_config_file = 'D:\\videos\\ar32\\2020_10_09\\ar32_10_09_20Face3-Arash-2020-10-10\\config.yaml';

## Label the extracted frames
Only videos in the config file can be used to extract the frames. Extracted labels for each video are stored in the project directory under the subdirectory **'labeled-data'**. Each subdirectory is named after the name of the video. The toolbox has a labeling toolbox which could be used for labeling. 

In [ ]:
%gui wx

deeplabcut.label_frames(path_config_file)

**Check the labels**

Checking if the labels were created and stored correctly is beneficial for training, since labeling is one of the most critical parts for creating the training dataset. The DeepLabCut toolbox provides a function `check\_labels'  to do so. It is used as follows:

In [ ]:

deeplabcut.check_labels(path_config_file) #this creates a subdirectory with the frames + your labels

## Reminder: Build your skeleton connections before you create a training set!


In [ ]:
deeplabcut.SkeletonBuilder(path_config_file)

If the labels need adjusted, you can use the refinement GUI to move them around! Check that out below.

## Create a training dataset
This function generates the training data information for DeepCut (which requires a mat file) based on the pandas dataframes that hold label information. The user can set the fraction of the training set size (from all labeled image in the hd5 file) in the config.yaml file. While creating the dataset, the user can create multiple shuffles. 

After running this script the training dataset is created and saved in the project directory under the subdirectory **'training-datasets'**

This function also creates new subdirectories under **dlc-models** and appends the project config.yaml file with the correct path to the training and testing pose configuration file. These files hold the parameters for training the network. Such an example file is provided with the toolbox and named as **pose_cfg.yaml**.

Now it is the time to start training the network!

In [ ]:
deeplabcut.create_training_dataset(path_config_file)

In [ ]:
deeplabcut.create_training_model_comparison(path_config_file, num_shuffles=1, net_types=['resnet_50'], augmenter_types=['default', 'imgaug'] )


## Start training  
### If yu want to use your GPU, you need to exit here and either work from the Docker container, your own TensorFlow installation in an Anaconda env

This function trains the network for a specific shuffle of the training dataset. 

In [ ]:

deeplabcut.train_network(path_config_file,shuffle=1,displayiters=1000,saveiters=1000)

## Start evaluating
This funtion evaluates a trained model for a specific shuffle/shuffles at a particular state or all the states on the data set (images)
and stores the results as .csv file in a subdirectory under **evaluation-results**

In [ ]:
deeplabcut.evaluate_network(path_config_file,Shuffles=[1], plotting=False)

## Start Analyzing videos
This function analyzes the new video. The user can choose the best model from the evaluation results and specify the correct snapshot index for the variable **snapshotindex** in the **config.yaml** file. Otherwise, by default the most recent snapshot is used to analyse the video.

The results are stored in hd5 file in the same directory where the video resides. 

In [3]:

path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_R-Arash-2020-02-28\\config.yaml';
#path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_L-Arash-2020-03-01\\config.yaml';
#path_config_file = 'E:\\movies\\ar2breathing\\10_08_19\\ar2breathing_10_08_19-Arash-2020-03-26\\config.yaml'
path_config_file = 'E:\\movies\\ar2breathing\\10_08_19\\ar2breathing_10_08_19LR_R-Arash-2020-04-30\\config.yaml'
path_config_file = 'D:\\videos\\ar21\\2020-07-06\\ar21_07_06_20Face2-Arash-2020-07-07\\config.yaml';
path_config_file = 'D:\\Dropbox\\Notebook\\ar19_09_15_20Face3-Arash-2020-09-16\\config.yaml';
path_config_file = 'D:\\Dropbox\\Notebook\\ar19_09_15_20Face3-Arash-2020-09-16\\config.yaml';
path_config_file = 'D:\\videos\\ar32\\2020_10_09\\ar32_10_09_20Face3-Arash-2020-10-10\\config.yaml';
import os

Mainfolder = 'E:\\movies\\ar2breathing\\10_06_19'
Mainfolder = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019'
Mainfolder = 'D:\\rat_movies_SC\\ar19tear19salin\\10_08_19'
Mainfolder = 'E:\\movies\\MUSCIMOL\\ar19tear19salin\\10_08_19'
Mainfolder = 'D:\\videos\\ar32\\2020_10_11'
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
#text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('R.avi') and not f.endswith('L.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
#text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]

#text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.mp4') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]

#path_config_file = 'F:\\ar19muscimol500ug500nlrightside\\10_10_19\\ar19_10102019-Fassihi-2019-11-12\\config.yaml';


#path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_L-Arash-2020-03-01\\config.yaml'
##text_files = ['']
#text_files = 'E:\\movies\\ar2breathing\\10_08_19\\20-18-27.avi'

print(text_files)
#deeplabcut.analyze_videos(path_config_file,text_files[1:len(text_files)],shuffle=1, save_as_csv=True)
#deeplabcut.analyze_videos(path_config_file,text_files[3],shuffle=1, save_as_csv=True)
deeplabcut.analyze_videos(path_config_file,text_files,shuffle=1, save_as_csv=True)

['D:\\videos\\ar32\\2020_10_11\\1.avi', 'D:\\videos\\ar32\\2020_10_11\\10.avi', 'D:\\videos\\ar32\\2020_10_11\\100.avi', 'D:\\videos\\ar32\\2020_10_11\\101.avi', 'D:\\videos\\ar32\\2020_10_11\\102.avi', 'D:\\videos\\ar32\\2020_10_11\\103.avi', 'D:\\videos\\ar32\\2020_10_11\\104.avi', 'D:\\videos\\ar32\\2020_10_11\\105.avi', 'D:\\videos\\ar32\\2020_10_11\\106.avi', 'D:\\videos\\ar32\\2020_10_11\\107.avi', 'D:\\videos\\ar32\\2020_10_11\\108.avi', 'D:\\videos\\ar32\\2020_10_11\\109.avi', 'D:\\videos\\ar32\\2020_10_11\\11.avi', 'D:\\videos\\ar32\\2020_10_11\\110.avi', 'D:\\videos\\ar32\\2020_10_11\\111.avi', 'D:\\videos\\ar32\\2020_10_11\\112.avi', 'D:\\videos\\ar32\\2020_10_11\\113.avi', 'D:\\videos\\ar32\\2020_10_11\\114.avi', 'D:\\videos\\ar32\\2020_10_11\\115.avi', 'D:\\videos\\ar32\\2020_10_11\\116.avi', 'D:\\videos\\ar32\\2020_10_11\\117.avi', 'D:\\videos\\ar32\\2020_10_11\\118.avi', 'D:\\videos\\ar32\\2020_10_11\\119.avi', 'D:\\videos\\ar32\\2020_10_11\\12.avi', 'D:\\videos\\ar32\\2

Starting to analyze %  D:\videos\ar32\2020_10_11\113.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\114.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\115.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\116.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\117.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\118.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\119.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\12.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\120.avi
D:\videos\ar32\2020_10_11  already exists!
Starting to analyze %  D:\videos\ar32\2020_10_11\121.avi
D:\videos\ar32\2020_10_11  already exists!
S

  2%|█▎                                                                               | 10/600 [00:06<06:30,  1.51it/s]


Starting to analyze %  D:\videos\ar32\2020_10_11\28.avi
D:\videos\ar32\2020_10_11  already exists!
Loading  D:\videos\ar32\2020_10_11\28.avi
Duration of video [s]:  15.0 , recorded with  40.0 fps!
Overall # of frames:  600  found with (before cropping) frame dimensions:  1448 470
Starting to extract posture


  2%|█▎                                                                               | 10/600 [00:00<00:06, 86.28it/s]


...

  2%|█▎                                                                               | 10/600 [00:00<00:09, 59.33it/s]


Starting to analyze %  D:\videos\ar32\2020_10_11\99.avi
D:\videos\ar32\2020_10_11  already exists!
Loading  D:\videos\ar32\2020_10_11\99.avi
Duration of video [s]:  15.0 , recorded with  40.0 fps!
Overall # of frames:  600  found with (before cropping) frame dimensions:  1448 470
Starting to extract posture


  2%|█▎                                                                               | 10/600 [00:00<00:08, 69.63it/s]


The videos are analyzed. Now your research can truly start! 
 You can create labeled videos with 'create_labeled_video'
If the tracking is not satisfactory for some videos, consider expanding the training set. You can use the function 'extract_outlier_frames' to extract a few representative outlier frames.


'DLC_resnet50_ar32_10_09_20Face3Oct10shuffle1_205000'

## Track non tracked files


In [ ]:
# Track non tracked files 
import os
path_config_file = 'F:\\ar2breathing\\10_06_19\\ar2_10062019_ii-Fassihi-2019-10-07\\config.yaml';

Mainfolder = 'Y:\\movies_Rat_SC_project\\ar2breathing\\10_06_19'
text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('breathing.avi')]
text_files2 = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('L.avi') ]
#print(text_files)
#%whos

text_files3 = [f.replace("L.", ".") for f in text_files2 ]
one_not_two = set(text_files) - set(text_files3)
one_not_two = 'Y:\\movies_Rat_SC_project\\ar2breathing\\10_06_19\\22-25-05.avi'
print(one_not_two)
deeplabcut.analyze_videos(path_config_file,one_not_two,shuffle=1, save_as_csv=True)

# Track Tracklets and then filter for multiple animals (maybe whiskers)

In [ ]:
Mainfolder = 'D:\\videos\\ar21\\2020_07_08\\'
import os

text_files = [os.path.join(Mainfolder,f) for f in os.listdir(Mainfolder) if f.endswith('.avi') and not f.endswith('L.avi') and not f.endswith('R.avi') and not f.endswith('videopoints.avi') and not f.endswith('videopoints.avi')]
print(text_files)
deeplabcut.convert_detections2tracklets(path_config_file, text_files, videotype='avi',
                                                    shuffle=1, trainingsetindex=0, track_method='skeleton')


In [ ]:
deeplabcut.refine_tracklets(path_config_file, pickle_or_h5_file, videofile_path, min_swap_len=2, min_tracklet_len=2, trail_len=50)


# filter the results using median model


In [4]:
deeplabcut.filterpredictions(path_config_file,text_files)


Filtering with median model D:\videos\ar32\2020_10_11\1.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\10.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\100.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\101.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\102.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\103.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\104.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\105.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\106.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\107.avi
Saving filtered csv poses!
Filtering with median model D:\videos\ar32\2020_10_11\108.avi
Saving filtered csv poses!
Filtering with median mo

No unfiltered data file found in D:\videos\ar32\2020_10_11 for video 7 and scorer DLC_resnet50_ar32_10_09_20Face3Oct10shuffle1_205000.
Filtering with median model D:\videos\ar32\2020_10_11\70.avi
No unfiltered data file found in D:\videos\ar32\2020_10_11 for video 70 and scorer DLC_resnet50_ar32_10_09_20Face3Oct10shuffle1_205000.
Filtering with median model D:\videos\ar32\2020_10_11\71.avi
No unfiltered data file found in D:\videos\ar32\2020_10_11 for video 71 and scorer DLC_resnet50_ar32_10_09_20Face3Oct10shuffle1_205000.
Filtering with median model D:\videos\ar32\2020_10_11\72.avi
No unfiltered data file found in D:\videos\ar32\2020_10_11 for video 72 and scorer DLC_resnet50_ar32_10_09_20Face3Oct10shuffle1_205000.
Filtering with median model D:\videos\ar32\2020_10_11\73.avi
No unfiltered data file found in D:\videos\ar32\2020_10_11 for video 73 and scorer DLC_resnet50_ar32_10_09_20Face3Oct10shuffle1_205000.
Filtering with median model D:\videos\ar32\2020_10_11\74.avi
No unfiltered da

# plot trajectories

In [ ]:
deeplabcut.plot_trajectories(path_config_file,text_files,filtered = True)


## Create labeled video
This funtion is for visualiztion purpose and can be used to create a video in .mp4 format with labels predicted by the network. This video is saved in the same directory where the original video resides. 

In [ ]:
# first we create a vector of destination file names
import os 

dirname = os.path.dirname(text_files[1]) 

thesenames= [os.path.basename(f)  for f in text_files] 
text_files2 = [os.path.join( os.path.join(os.path.dirname(text_files[1]),'labled'),f) for f in thesenames ]  

#os.mkdir
if not (os.path.isdir(os.path.join(os.path.dirname(text_files[1]),'labled'))):
 os.mkdir(os.path.join(os.path.dirname(text_files[1]),'labled'))
# Print the directory name   
print(text_files2[1]) 
for f in text_files:

#deeplabcut.create_labeled_video(path_config_file,videofile,save_frames=True) # my_new_list was created in prevouse cell as all videos in the folder 
#deeplabcut.create_labeled_video(path_config_file,[text_files[0],text_files2[0]],save_frames=True)
 deeplabcut.create_labeled_video(path_config_file,f,save_frames=False)

## Extract outlier frames [This is important step actually]
This is an optional step and is used only when the evaluation results are poor i.e. the labels are incorrectly predicted. In such a case, the user can use the following function to extract frames where the labels are incorrectly predicted. Make sure to provide the correct value of the "iterations" as it will be used to create the unique directory where the extracted frames will be saved.

In [ ]:
#video=['D:\\rat_movies_SC\\ar17record\\02_27_19\\analyzed\\18-37-27.avi']
#video=['D:\\rat_movies_SC\\ar15record\\02_27_19\\17-33-50video.mp4']
#video=['D:\\Dropbox\\ar17\\02_25_19\\22-37-23.avi']
video = ['D:\\movies_Rat_SC_project\\ar1\\05_24_19\\11-09-14.avi']
video = ['D:\\movies_Rat_SC_project\\ar20\\06_20_19\\17-19-45.avi','D:\\movies_Rat_SC_project\\ar20\\06_20_19\\17-31-05.avi','D:\\movies_Rat_SC_project\\ar20\\06_20_19\\17-30-40.avi']
#15-17-08v
video = ['D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-27-18.avi','D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-24-47.avi']
video = ['D:\\rat_movies_SC\\ar19tear19salin\\10_08_19\\00-24-26video.mp4']
video = ['E:\\movies\\ar2breathing\\10_06_19\\21-57-55L.avi']
video = text_files[0]

#path_config_file = 'D:\\rat_movies_SC\\ar19muscimol\\10_10_2019\\ar19muscimol10-10-19_LR_L-Arash-2020-03-01\\config.yaml';

#path_config_file = 'D:\\rat_movies_SC\\ar17record\\02_27_19\\sc_ar17record_02272019_3-Fassihi-2019-04-28\\config.yaml'
#path_config_file = 'D:\\rat_movies_SC\\ar15record\\02_27_19\\sc_ar15record_02272019-Fassihi-2019-03-12\\config.yaml'
#path_config_file = 'D:\\movies_Rat_SC_project\\ar1\\05_24_19\\sc_aa1_05242019-Fassihi-2019-05-30\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar3\\06_07_19\\sc_aa3_06072019-Fassihi-2019-06-10\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar14\\06_18_19\\sc_ar14_06182019-Fassihi-2019-06-19\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
print(path_config_file)

#deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',p_bound=.6)
#deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',comparisonbodyparts =  ['nose','snout'],p_bound=.6)
deeplabcut.extract_outlier_frames(path_config_file,video,outlieralgorithm='uncertain',p_bound=.9)



In [ ]:
print(video[2])


In [ ]:

path_config_file = 'D:\\rat_movies_SC\\ar19\\02_07_19\\sc_ar19_02072019-Fassihi4-2019-02-13\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

import os
Mainfolder = 'D:\\rat_movies_SC\\ar19\\02_07_19'


f = os.listdir(Mainfolder)
text_files = [f for f in os.listdir(Mainfolder) if f.endswith('.avi')]


my_list = text_files
thisApen = Mainfolder+'kir'
thisApen = thisApen.replace("kir", "\\")
string = thisApen
my_new_list = [ string + x for x in my_list]

deeplabcut.extract_outlier_frames(path_config_file,my_new_list)

## Refine Labels [optional step]
Following the extraction of outlier frames, the user can use the following function to move the predicted labels to the correct location. Thus augmenting the training dataset.

In [ ]:
#path_config_file = 'D:\\movies_Rat_SC_project\\ar1\\05_24_19\\sc_aa1_05242019-Fassihi-2019-05-30\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar3\\06_07_19\\sc_aa3_06072019-Fassihi-2019-06-10\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

#import matplotlib
#print(matplotlib.__version__)
%gui wx
#path_config_file = 'D:\\rat_movies_SC\\ar15record\\02_27_19\\sc_ar15record_02272019-Fassihi-2019-03-12\\config.yaml'
#print(path_config_file)
deeplabcut.refine_labels(path_config_file)

In [ ]:
#Once all folders are relabeled, check them and advance. See how to check labels, above!
#path_config_file = 'D:\\movies_Rat_SC_project\\ar1\\05_24_19\\sc_aa1_05242019-Fassihi-2019-05-30\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)
#path_config_file = 'D:\\movies_Rat_SC_project\\ar3\\06_07_19\\sc_aa3_06072019-Fassihi-2019-06-10\\config.yaml' #Enter the path of the config file that was just created from the above step (check the folder)

deeplabcut.merge_datasets(path_config_file)

## Create a new iteration of training dataset [optional step]
Following the refine labels, append these frames to the original dataset to create a new iteration of training dataset.

In [ ]:
list_of_ints = list(range(2))
print(list_of_ints)
deeplabcut.create_training_dataset(path_config_file,Shuffles= list_of_ints) ;

## Plot the trajectories of the analyzed videos (optional)
This function plots the trajectories of all the body parts across the entire video. Each body part is identified by a unique color.

In [ ]:
%matplotlib notebook #for making interactive plots.
deeplabcut.plot_trajectories(path_config_file,my_new_list)

